In [1]:
import numpy as np
import pandas as pd
import pickle
import os.path
import math
import sys
import csv
import matplotlib.pyplot as plt
from sklearn.externals import joblib
import scipy.integrate as spi
import pylab as pl

/home/ec2-user/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ec2-user/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Load Data

In [2]:
n_bytes = 2**31
max_bytes = 2**31 - 1

def load_pickle(file_path):
    
    bytes_in = bytearray(0)
    input_size = os.path.getsize(file_path)
    with open(file_path, 'rb') as f_in:
        for _ in range(0, input_size, max_bytes):
            bytes_in += f_in.read(max_bytes)
        return pickle.loads(bytes_in)
    
users = load_pickle("data/users.dat")
friends = load_pickle("data/friends.dat")
network = load_pickle("data/network.dat")
data = load_pickle("data/30_data.dat")
new_data = load_pickle("data/30data_new.dat")

In [3]:
users.head()

## friends
# for key, value in friends.items():
#     print(key, value)

# network.head()

,user,user_id,time_lapsed,followers_count,friends_count,mention_and_reply,source_candidates,source_index,seed_index,generation,time_since_seed
0,davelackie,100766356,0.00,143119,4643,[],"[3, 5, 6, 7, 10, 11, 14, 15, 19, 20, 24, 74, 8...",None,0,0,0
1,alexanderskhan,3186545203,0.15,753,3428,[0],[0],0,0,1,0.15
2,consiglierela,4134992843,0.23,2161,396,[0],"[0, 12, 21, 32, 48, 748, 819, 972, 2358, 2778,...",0,0,1,0.23
3,ImpactPrincess,1955472014,0.63,242,642,[0],"[0, 115, 577]",0,0,1,0.63
4,Damek0Masca,406301175,1.13,116,239,[],[],None,4,0,0


## The seeds

In [4]:
record_num = users.describe().iloc[0,0]
record_num = int(record_num)
for i in range(record_num):
    record = users.loc[i,"generation"]
    if record == 0:
        seeds = users.loc[i,:]
        #print(seeds)

## Initial Adjacency Matrix

In [5]:
node_num = users.shape[0]
# node_num

In [6]:
ad_matrix=[[0 for col in range(node_num)] for row in range(node_num)]
# ad_matrix[0][0:5]

In [7]:
for i in range(node_num):
    length = len(users.loc[i,"source_candidates"])
    for j in range(length):
        parents = users.loc[i,"source_candidates"][j]
        ad_matrix[i][parents] = 1
#ad_matrix[0:1][0:100]

## Load the model

In [8]:
def loadModel(self):
    read_file = self.model_name + ".pickle.dat"
    self.model = joblib.load(read_file)
    return self.model

model = joblib.load("predictor_1.pickle")
# model

## Spread simulation ——A_Simple_Trial

In [15]:
users_new = pd.DataFrame(columns=["user","candidates","SIR","generations"])
users_new["user"] = users["user"]
users_new["candidates"] = users["source_candidates"]
users_new["SIR"] =0
users_new.head()

#set 0,2,3 as seeds
#in SIR,0=uninfected, 1=infected, 2=immunity
infec=0.5
N= 5

# users_new["SIR"] = "Null"
# users_new["generations"] ="Null"

users_new.loc[0,"SIR"]= 1
users_new.loc[0,"generations"]=0
users_new.loc[2,"SIR"]= 1
users_new.loc[2,"generations"]=0
users_new.loc[3,"SIR"]= 1
users_new.loc[3,"generations"]=0
for n in range(N):
    for i in range(record_num):
        if users_new.loc[i,"generations"] == n and users_new.loc[i,"SIR"] == 1:
            y=users_new.loc[i,"candidates"]
            for j in range(len(y)):
                parents = users_new.loc[i,"candidates"][j]
                if np.random.uniform(0,1) > infec and users_new.loc[parents,"SIR"] ==0:
                    users_new.loc[parents,"SIR"] = 1
                    users_new.loc[parents,"generations"] = n+1
            users_new.loc[i,"SIR"]=2
users_new.head()

,user,candidates,SIR,generations
0,davelackie,"[3, 5, 6, 7, 10, 11, 14, 15, 19, 20, 24, 74, 8...",2,0
1,alexanderskhan,[0],0,NaN
2,consiglierela,"[0, 12, 21, 32, 48, 748, 819, 972, 2358, 2778,...",2,0
3,ImpactPrincess,"[0, 115, 577]",2,0
4,Damek0Masca,[],0,NaN


## Simulation

In [10]:
data.shape[0]

226984

In [11]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226984 entries, 0 to 46
Data columns (total 74 columns):
Nw_averageNeighborDegree                      226984 non-null float64
Nw_averageNeighborDegree-1                    226984 non-null float64
Nw_averageNeighborDegree0                     226984 non-null float64
Nw_betweennessCentrality                      226984 non-null float64
Nw_betweennessCentrality-1                    226984 non-null float64
Nw_betweennessCentrality0                     226984 non-null float64
Nw_closenessCentrality                        226984 non-null float64
Nw_closenessCentrality-1                      226984 non-null float64
Nw_closenessCentrality0                       226984 non-null float64
Nw_degreeCentrality                           226984 non-null float64
Nw_degreeCentrality-1                         226984 non-null float64
Nw_degreeCentrality0                          226984 non-null float64
Nw_eigenvectorCentrality                      226984 

In [12]:
valid_X = data.drop(["label","kOutMin"],axis=1)
valid_X
infec = model.predict_proba(valid_X)
infec[0:4]  #左闭右开 右大1左大0
infected = []

for line in infec:
    if line[1]>0.5:
        infected.append(1)
    else:
        infected.append(0)
len(infected)

226984

In [18]:
len(infec)

TypeError: object of type 'float' has no len()

In [14]:
users_new = pd.DataFrame(columns=["user","candidates","SIR","generations"])
users_new["user"] = users["user"]
users_new["candidates"] = users["source_candidates"]
users_new["SIR"] =0
users_new.head()

#set 0,2,3 as seeds
#in SIR,0=uninfected, 1=infected, 2=immunity
N= 47
users_new.loc[0,"SIR"]= 1
users_new.loc[0,"generations"]=0
users_new.loc[2,"SIR"]= 1
users_new.loc[2,"generations"]=0
users_new.loc[3,"SIR"]= 1
users_new.loc[3,"generations"]=0
for n in range(N):
    for i in range(record_num):
        if users_new.loc[i,"generations"] == n and users_new.loc[i,"SIR"] == 1:
            y=users_new.loc[i,"candidates"]
            for j in range(len(y)):
                parents = users_new.loc[i,"candidates"][j]
                locate = n+47*5011
                print(locate)
                if infected[locate]==1 :
                    users_new.loc[parents,"SIR"] = 1
                    users_new.loc[parents,"generations"] = n+1
            users_new.loc[i,"SIR"]=2
users_new.head(20)

235517


IndexError: list index out of range

##  SIR_MODEL

In [ ]:
#### SIR_MODEL #### 
beta=1.6667
gamma=1
TS=1.0
ND=40.0
S0=1-1e-6
I0=1e-6
INPUT = (S0, I0, 0.0)
 
def diff_eqs(INP,t):
    '''The main set of equations'''
    Y=np.zeros((3))
    V = INP
    Y[0] = - beta * V[0] * V[1]
    Y[1] = beta * V[0] * V[1] - gamma * V[1]
    Y[2] = gamma * V[1]
    return Y   # For odeint
 
t_start = 0.0; t_end = ND; t_inc = TS
t_range = np.arange(t_start, t_end+t_inc, t_inc)
RES = spi.odeint(diff_eqs,INPUT,t_range)
 
#print(RES)
 
#Ploting
pl.subplot(111)
pl.plot(RES[:,1], '-r', label='Infectious')
pl.plot(RES[:,0], '-g', label='Susceptibles')
pl.plot(RES[:,2], '-k', label='Recovereds')
pl.legend(loc=0)
pl.title('SIR_Model.py')
pl.xlabel('Time')
pl.ylabel('Infectious Susceptibles and Recovereds')
pl.show()

    